In [1]:
sample = "E14.5_embryo_dox_E10.5"
locus = "CA"
called_bc_file = f"BST_output/{sample}_{locus}/02.Umi2Gene/features.tsv"
R1_file  = f"./slim_fastq/{sample}_{locus}_R1.fastq.gz"
R2_file  = f"./slim_fastq/{sample}_{locus}_R2.fastq.gz"

sc10xv3_cell_barcode_whitelist = "./10xV3_barcodes.txt.gz" ## {carlin_dir}/cfg/10xV3_barcodes.txt.gz

In [ ]:
if locus == "CA":
    template = "cCARLIN"
elif locus == "TA":
    template = "Tigre_2022_v2"
elif locus == "RA":
    template = "Rosa_v2"
else:
    raise ValueError(f"Invalid locus: {locus}. locus must be ['CA','TA','RA']")

In [2]:
## Matlab -> python. uisng clone_id identified by the python-version of the snakemake_DARLIN pipeline, 
## convert to raw fastq file, then run matlab version to get the alignment results.
import os
import gzip
import random
import pandas as pd
# from mosaiclineage import DARLIN as dar

In [3]:
def gen_umi(length: int) -> str:
    bases = ['A', 'T', 'C', 'G']
    umi_sequence = ''.join(random.choice(bases) for _ in range(length))
    return umi_sequence

def gen_base_qual_score(length: int) -> str:
    qual_scores = [chr(random.randint(30, 40) + 33) for _ in range(length)]
    return ''.join(qual_scores)

def gen_10xv3_R1(seq_id: str, cell_barcode: str) -> str:
    if len(cell_barcode) != 16:
        raise ValueError("Cell barcode must be 16 bases long")
    umi_sequence = gen_umi(12)
    sequence = cell_barcode + umi_sequence
    qual_score = gen_base_qual_score(28)
    fastq_read = f"@{seq_id}\n{sequence}\n+\n{qual_score}"
    return fastq_read

def gen_10xv3_R1_fastq(cell_barcodes: list, filename: str) -> None:
    """
    Generates a compressed FASTQ file containing reads for each cell barcode provided.

    Parameters:
        cell_barcodes (list): A list of cell barcodes, each 16 bases long.
        filename (str): The output filename for the compressed FASTQ file (e.g., "R1_reads.fastq.gz").

    Raises:
        ValueError: If any cell barcode in the list is not 16 bases long.

    The function creates a gzip-compressed FASTQ file where each read is generated using the provided cell barcodes.
    The sequence ID for each read is derived from the filename prefix and the index of the barcode in the list.
    """
    prefix = filename.split('/')[-1].replace("_R1.fastq.gz", "")
    with gzip.open(filename, 'wt') as f:
        for i, barcode in enumerate(cell_barcodes):
            if len(barcode) != 16:
                raise ValueError(f"Cell barcode at index {i} must be 16 bases long")
            seq_id = f"{prefix}_{i+1}"
            fastq_read = gen_10xv3_R1(seq_id, barcode)
            f.write(fastq_read + '\n')

def gen_10xv3_R2_fastq(sequences: list, filename: str) -> None:
    """
    Generates a compressed FASTQ file containing R2 reads for each sequence provided.

    Parameters:
        sequences (list): A list of sequences to be included in the R2 reads.
        filename (str): The output filename for the compressed FASTQ file (e.g., "R2_reads.fastq.gz").
    """
    prefix = filename.split('/')[-1].replace("_R2.fastq.gz", "")
    with gzip.open(filename, 'wt') as f:
        for i, sequence in enumerate(sequences):
            seq_id = f"{prefix}_{i+1}"
            qual_score = gen_base_qual_score(len(sequence))
            fastq_read = f"@{seq_id}\n{sequence}\n+\n{qual_score}"
            f.write(fastq_read + '\n')

def gen_10xv3_fastq(locus: str, cell_bc: list, clone_bc: list, R1_file: str, R2_file: str) -> None:
    if locus.startswith("Tigre"):
        seq_5prime = 'GCTCGGTACCTCGCGAA'
        seq_3prime = 'GTCTTGTCGGTGCCT'
    elif locus.startswith("cCARLIN"):
        seq_5prime = 'GAGCTGTACAAGTAAGCGGC'
        seq_3prime = 'AGAATTCTAACTAGAGCTCGCTGATCAGCCT'
    elif locus.startswith("Rosa"):
        seq_5prime = 'ATGTACAAGTAAAGCGGCC'
        seq_3prime = 'GTCTGCTGTGTGCCT'
    else:
        raise ValueError(f"Invalid locus: {locus}. locus must start with ['cCARLIN','Tigre','Rosa']")
    clone_bc = [seq_5prime + i + seq_3prime for i in clone_bc]
    R2_dir = os.path.dirname(R2_file)
    if not os.path.exists(R2_dir):
        os.makedirs(R2_dir)
    gen_10xv3_R1_fastq(cell_bc,  R1_file)
    gen_10xv3_R2_fastq(clone_bc, R2_file)


In [5]:
#### main ####
## load sc10xv3 cell barcode whitelist
with gzip.open(sc10xv3_cell_barcode_whitelist, 'rt') as f:
    cell_barcodes = [line.strip() for line in f]

## load DARLIN output
data_df = pd.read_csv(called_bc_file, sep='\t', header=None, names=['clone_id', 'clone_bc', 'type'])
n_lines = len(data_df['clone_id'])
n_uniq_clone_bcs = len(data_df['clone_id'].unique())
msg = f'{n_lines} lines. {n_uniq_clone_bcs} unique clone barcodes'
print(msg)

## generate 10xv3 fastq
clone_barcodes = data_df['clone_id']
cell_barcodes = random.sample(cell_barcodes, n_uniq_clone_bcs)

gen_10xv3_fastq(
    locus    = template, 
    cell_bc  = cell_barcodes, 
    clone_bc = clone_barcodes,
    R1_file = R1_file,
    R2_file = R2_file
)

875 lines. 875 unique clone barcodes


In [8]:
!zcat slim_fastq/{sample}_{locus}_R1.fastq.gz | head -10

@E14.5_embryo_dox_E10.5_CA_1
CGAGTGCGTTCGGAGATGTGAGCATGCT
+
GGHEGIDFBE@ABDCFBB?HDA?BFGHC
@E14.5_embryo_dox_E10.5_CA_2
GCTGGGTGTATTGGTCGACCGGGGTCGA
+
IIEFBG@IBHF?DIGIEEBFD?FIBICF
@E14.5_embryo_dox_E10.5_CA_3
TTATTGCCAGACCTATACGTCGTCACTC

gzip: stdout: Broken pipe


In [7]:
!zcat slim_fastq/{sample}_{locus}_R2.fastq.gz | head -10

@E14.5_embryo_dox_E10.5_CA_1
GAGCTGTACAAGTAAGCGGCCGTGGGAGCTAGAATTCTAACTAGAGCTCGCTGATCAGCCT
+
@?AAEHCAI?EHC@CEAIGIIDFED@BE?GADABGGA??DD@?E@AD@DFBGEABC?DGIC
@E14.5_embryo_dox_E10.5_CA_2
GAGCTGTACAAGTAAGCGGCCGATGGGAGCTAGAATTCTAACTAGAGCTCGCTGATCAGCCT
+
?CGAA?BD?DH@GEIEIBHBHBCD@AH@FEBCGEGH?H???E@EF?HEHDBAIE?FADBF@D
@E14.5_embryo_dox_E10.5_CA_3
GAGCTGTACAAGTAAGCGGCCGCCGGACTGCACGAGAATTCTAACTAGAGCTCGCTGATCAGCCT

gzip: stdout: Broken pipe
